<a href="https://colab.research.google.com/github/Elizav555/Data_Mining/blob/main/Distances/Distances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://www.thegamer.com/horizon-forbidden-west-review-ps5/
read://https_www.thegamer.com/?url=https%3A%2F%2Fwww.thegamer.com%2Fhorizon-forbidden-west-review-ps5%2F



In [94]:
import nltk
import requests
import string
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
url = 'https://www.thegamer.com/horizon-forbidden-west-review-ps5/'
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text)
root = soup.body
root_childs = [e.name for e in root.descendants if e.name is not None]
#print(set(root_childs))

In [96]:
textTags = soup.find_all(['p','h1','h2','h3','h4','label','a','ul','span','li','button'])
whole_text = ' '.join(map(lambda it: it.text, textTags)) 
all_words = word_tokenize(whole_text)
#print(all_words)
print('Number of all words: ',len(all_words))

Number of all words:  2712


In [97]:
def process_whole_text(all_words_set):
  symb_to_remove = stopwords.words('english') +  list(string.punctuation) +  list(string.digits) + list('’')
  text_cleared = [word for word in all_words_set if word not in symb_to_remove]
  text_cleared = [word for word in text_cleared if not word.isdigit()]
  #text_cleared = set(text_cleared)
  #print(text_cleared)
  return text_cleared

horizon_article = process_whole_text(all_words)
print('Number of cleared words: ',len(horizon_article))

Number of cleared words:  1701


https://keywordgenerator.net

Сервис, откуда я взяла слова для топиков

In [98]:
def process_key_words(file_name ):
  dataset = pd.read_csv(file_name)
  dataset = dataset['Keyword']
  words = dataset.values.tolist()
  words = list(map(lambda it: it.split(' ') ,words))
  words_set = list()
  for words in words:
    words_set+=words
  words_set = set(words_set)
  words_set-=set(['']) 
  words_set = process_whole_text(words_set)
  return set(words_set)

In [99]:
news_words = process_key_words('news.csv')
science_words = process_key_words('science.csv')
shopping_words = process_key_words('shopping.csv')
sport_words = process_key_words('sport.csv')
# print('News: ',news_words,'\n', len(news_words))
# print('Science: ',science_words,'\n', len(science_words))
# print('Shopping: ',shopping_words,'\n', len(shopping_words))
# print('Sport: ',sport_words,'\n', len(sport_words))
print('News: ', len(news_words))
print('Science: ', len(science_words))
print('Shopping: ', len(shopping_words))
print('Sport: ', len(sport_words))

News:  756
Science:  873
Shopping:  829
Sport:  723


In [100]:
def parse_from_dictionary(url):
  response_dictionary = requests.get(url)
  soup_dictionary = BeautifulSoup(response_dictionary.text)
  textTags_dictionary = soup_dictionary.find_all('a',class_='btnpopover')
  whole_text_dictionary = ' '.join(map(lambda it: it.text, textTags_dictionary)) 
  all_words_dictionary = word_tokenize(whole_text_dictionary)
  all_words_dictionary = process_whole_text(all_words_dictionary)
  return(set(all_words_dictionary))


https://www.niftyword.com/related-words/science/ связанные слова

In [102]:
news_words = news_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/news/'))
science_words = science_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/science/'))
shopping_words= shopping_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/shopping/'))
sport_words =sport_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/sport/'))
# print('News: ',news_words,'\n', len(news_words))
# print('Science: ',science_words,'\n', len(science_words))
# print('Shopping: ',shopping_words,'\n', len(shopping_words))
# print('Sport: ',sport_words,'\n', len(sport_words))
print('News: ', len(news_words))
print('Science: ', len(science_words))
print('Shopping: ', len(shopping_words))
print('Sport: ', len(sport_words))

News:  2558
Science:  3477
Shopping:  3498
Sport:  3133


In [103]:
def jaccard_fun(a,b,topic):
  a= set(a)
  b=set(b)
  shared = a.intersection(b)
  total = a.union(b)
  res = len(shared) / len(total)
  print('Topic: ',topic,', Shared: ',len(shared),', Total: ',len(total),', Similarity: ',res,', Distance: ',1-res)

In [104]:
jaccard_fun(horizon_article ,news_words,'News')
jaccard_fun(horizon_article ,science_words,'Science')
jaccard_fun(horizon_article ,shopping_words,'Shopping')
jaccard_fun(horizon_article ,sport_words,'Sport')

Topic:  News , Shared:  92 , Total:  3213 , Similarity:  0.028633675692499222 , Distance:  0.9713663243075008
Topic:  Science , Shared:  115 , Total:  4109 , Similarity:  0.02798734485276223 , Distance:  0.9720126551472378
Topic:  Shopping , Shared:  119 , Total:  4126 , Similarity:  0.028841492971400873 , Distance:  0.9711585070285991
Topic:  Sport , Shared:  120 , Total:  3760 , Similarity:  0.031914893617021274 , Distance:  0.9680851063829787


https://www.topendsports.com/world/lists/popular-sport/

In [105]:
response_dictionary = requests.get('https://www.topendsports.com/world/lists/popular-sport/')
soup_dictionary = BeautifulSoup(response_dictionary.text)
textTags_dictionary = soup_dictionary.find_all(['p','h1','li','a'])
whole_text_dictionary = ' '.join(map(lambda it: it.text, textTags_dictionary)) 
all_words_dictionary = word_tokenize(whole_text_dictionary)
#print(all_words_dictionary)
print('Number of all words: ',len(all_words_dictionary))

Number of all words:  16194


In [106]:
sport_article = process_whole_text(all_words_dictionary)
#print(sport_article)
print('Number of cleared words: ',len(sport_article))

Number of cleared words:  8502


In [107]:
jaccard_fun(sport_article ,news_words,'News')
jaccard_fun(sport_article ,science_words,'Science')
jaccard_fun(sport_article ,shopping_words,'Shopping')
jaccard_fun(sport_article ,sport_words,'Sport')

Topic:  News , Shared:  326 , Total:  5124 , Similarity:  0.06362217017954723 , Distance:  0.9363778298204528
Topic:  Science , Shared:  386 , Total:  5983 , Similarity:  0.06451612903225806 , Distance:  0.935483870967742
Topic:  Shopping , Shared:  412 , Total:  5978 , Similarity:  0.06891937102709936 , Distance:  0.9310806289729007
Topic:  Sport , Shared:  480 , Total:  5545 , Similarity:  0.08656447249774572 , Distance:  0.9134355275022543


In [108]:
data = [" ".join(horizon_article), " ".join(sport_article)," ".join(news_words)," ".join(science_words)," ".join(shopping_words)," ".join(sport_words)]
count_vectorizer = CountVectorizer()
vector_matrix = count_vectorizer.fit_transform(data)
tokens = count_vectorizer.get_feature_names_out()
tokens

array(['000', '0nline', '10', ..., 'zurich', 'zymology', 'zymurgy'],
      dtype=object)

In [109]:
vector_matrix_array = vector_matrix.toarray()
vector_matrix_array

array([[0, 0, 0, ..., 0, 0, 0],
       [3, 0, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [110]:
def create_dataframe(matrix, tokens):
    doc_names = ["horizon_article","sport_article","news_words","science_words","shopping_words","sport_words"]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

dataframe = create_dataframe(vector_matrix_array,tokens)
dataframe

,000,0nline,10,100,10th,11,11players,12,12th,145,...,zoological,zoologist,zoology,zoom,zourkhaneh,zp,zuoz,zurich,zymology,zymurgy
horizon_article,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sport_article,3,0,2,2,0,0,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
news_words,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
science_words,0,0,1,0,1,1,0,2,1,0,...,1,1,1,1,0,0,0,0,1,1
shopping_words,0,1,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sport_words,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,0


In [111]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,["horizon_article","sport_article","news_words","science_words","shopping_words","sport_words"])

,horizon_article,sport_article,news_words,science_words,shopping_words,sport_words
horizon_article,1.000000,0.252811,0.082712,0.069287,0.077076,0.078680
sport_article,0.252811,1.000000,0.075952,0.073653,0.074523,0.138580
news_words,0.082712,0.075952,1.000000,0.188529,0.281104,0.185942
science_words,0.069287,0.073653,0.188529,1.000000,0.239160,0.275769
shopping_words,0.077076,0.074523,0.281104,0.239160,1.000000,0.343177
sport_words,0.078680,0.138580,0.185942,0.275769,0.343177,1.000000


In [112]:
def jaccard_vector_fun(x, y,topic):
    intersect = 0
    union = 0
    for i,val in enumerate(x):
      if x[i]&y[i]!=0:
        intersect +=1
      if x[i]|y[i]!=0:
        union +=1
    res = intersect / union
    print('Topic: ',topic,', Shared: ',intersect,', Total: ',union,', Similarity: ',res,', Distance: ',1-res)

In [113]:
jaccard_vector_fun(vector_matrix_array[0] ,vector_matrix_array[2],'News')
jaccard_vector_fun(vector_matrix_array[0] ,vector_matrix_array[3],'Science')
jaccard_vector_fun(vector_matrix_array[0],vector_matrix_array[4],'Shopping')
jaccard_vector_fun(vector_matrix_array[0],vector_matrix_array[5],'Sport')

Topic:  News , Shared:  92 , Total:  3087 , Similarity:  0.029802397149335923 , Distance:  0.970197602850664
Topic:  Science , Shared:  99 , Total:  3996 , Similarity:  0.024774774774774775 , Distance:  0.9752252252252253
Topic:  Shopping , Shared:  114 , Total:  3996 , Similarity:  0.028528528528528527 , Distance:  0.9714714714714715
Topic:  Sport , Shared:  108 , Total:  3645 , Similarity:  0.02962962962962963 , Distance:  0.9703703703703703


In [114]:
jaccard_vector_fun(vector_matrix_array[1] ,vector_matrix_array[2],'News')
jaccard_vector_fun(vector_matrix_array[1] ,vector_matrix_array[3],'Science')
jaccard_vector_fun(vector_matrix_array[1],vector_matrix_array[4],'Shopping')
jaccard_vector_fun(vector_matrix_array[1],vector_matrix_array[5],'Sport')

Topic:  News , Shared:  241 , Total:  4507 , Similarity:  0.053472376303527844 , Distance:  0.9465276236964721
Topic:  Science , Shared:  253 , Total:  5385 , Similarity:  0.046982358402971214 , Distance:  0.9530176415970288
Topic:  Shopping , Shared:  285 , Total:  5371 , Similarity:  0.0530627443679017 , Distance:  0.9469372556320983
Topic:  Sport , Shared:  343 , Total:  4937 , Similarity:  0.06947538991290257 , Distance:  0.9305246100870974
